### Name Entity From Capital Letter Words

In [ ]:
#!pip install kneed

In [ ]:
import pandas as pd
import numpy as np
import re
from kneed import KneeLocator

#### Capital Letter Word And Lower Letter Word

In [ ]:
folder_name = "Dutch"

In [ ]:
df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name}/Result/Word/Capital/Not_Apply_Lower_Word_Merge_Result.csv") # Not apply lower technique
df

In [ ]:
df = df[df.iloc[:,1]>=5]
df

In [ ]:
def word_length(var):
    var = str(var)
    if len(var) <= 15:
        return var
    else:
        return "word is bigger than adjust value"

In [ ]:
df = pd.DataFrame(df.iloc[:,0].apply(lambda x: word_length(x)))

In [ ]:
df.drop(df[df.iloc[:,0] == "word is bigger than adjust value"].index, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
capital_word_list = []
lower_word_list = []
for i in df.iloc[:,0]:
    i = str(i)
    if i[0].isupper():
        capital_word_list.append(i)
    else:
        lower_word_list.append(i)

In [ ]:
df_capital = pd.DataFrame(capital_word_list)
df_capital.rename(columns={0:"word"}, inplace=True)
df_capital = pd.DataFrame(df_capital.iloc[:,0].apply(lambda x: x.capitalize()))
df_capital

In [ ]:
df_capital_to_lower = pd.DataFrame(df_capital.iloc[:,0].apply(lambda x : x.lower()))
df_capital_to_lower

In [ ]:
df_lower = pd.DataFrame(lower_word_list)
df_lower.rename(columns={0:"word"}, inplace=True)
df_lower

In [ ]:
df_lower_to_lower = pd.DataFrame(df_lower.iloc[:,0].apply(lambda x : x.lower()))
df_lower_to_lower

In [ ]:
capital_set = set(df_capital_to_lower.iloc[:,0])

In [ ]:
lower_set = set(df_lower_to_lower.iloc[:,0])

In [ ]:
df_all_capital = pd.DataFrame(capital_set.difference(lower_set)) # compare capital letter word and lower letter word
df_all_capital.rename(columns={0:"word"}, inplace=True)
#df_all_capital = pd.DataFrame(df_all_capital.iloc[:,0].apply(lambda x: x.capitalize()))
df_all_capital

In [ ]:
# Check result with all capital word
# df_merge = pd.merge(df_capital, df_all_capital, how="inner", on="word")
# df_merge.drop_duplicates(inplace=True)
# df_merge.reset_index(drop=True, inplace=True)
# df_merge

In [ ]:
df_all_capital.to_excel("All_Time_Capital_Letter_Words.xlsx", sheet_name="Word", index=False)

#### Sentence First Word Analysis

In [ ]:
df_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name}/Result/Sentence/Merge/Sentence_Merge.csv")
df_sent

In [ ]:
#df_sent = df_sent[df_sent.iloc[:,1]>=5]

In [ ]:
first_words = []
for i in df_sent.iloc[:,0]: # Select sentence first word
    words = re.findall("\w+", i, re.UNICODE)
    first_words.append(words[0])

In [ ]:
first_ser = pd.Series(first_words)
first_ser

In [ ]:
df_first = pd.DataFrame(first_ser.value_counts(ascending=False))
df_first.reset_index(inplace=True)
df_first.rename(columns={"index":"word", 0:"frequency"}, inplace=True)
df_first

In [ ]:
set_first = set(df_first.iloc[:,0])

In [ ]:
set_all_capital = set(df_all_capital.iloc[:,0])

In [ ]:
df_all_spec = pd.DataFrame(set_all_capital.difference(set_first)) # Capital letter word and first sentence word
df_all_spec.rename(columns={0:"word"}, inplace=True)
df_all_spec

In [ ]:
df_all_spec.to_excel("Special_Name_Exactly_Full.xlsx", sheet_name="Spec_Name", index=False)

#### Spec Name Knee Point

In [ ]:
df_word = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name}/Result/Word/Merge/Word_Merge.csv") # for frequency
df_word

In [ ]:
df_all_spec_freq = pd.merge(df_word, df_all_spec, how="inner", on="word") # frequency value
df_all_spec_freq

In [ ]:
total_freq_spec = df_all_spec_freq.iloc[:,1].sum()
total_freq_spec

In [ ]:
df_all_spec_freq["ratio"] = round(((df_all_spec_freq.iloc[:,1]/total_freq_spec)*100),7)
df_all_spec_freq

In [ ]:
df_all_spec_freq["cumul_ratio"] = np.cumsum(df_all_spec_freq.iloc[:,2])
df_all_spec_freq

In [ ]:
spec_kneedle = KneeLocator(x=df_all_spec_freq.cumul_ratio.index, y=df_all_spec_freq.cumul_ratio, S=1.0, curve="concave", direction="increasing")
spec_kneedle.plot_knee()

In [ ]:
print(round(spec_kneedle.knee_y))

In [ ]:
knee_point_spec = round(spec_kneedle.knee_y)

In [ ]:
df_spec_knee = df_all_spec_freq[df_all_spec_freq.cumul_ratio <= knee_point_spec]
df_spec_knee

In [ ]:
df_spec_knee.to_excel("Special_Name_Exactly_Knee.xlsx", sheet_name="Spec_Name", index=False)

#### First 100 000 Word And Spec Name Analysis

In [ ]:
df_word = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name}/Result/Word/Merge/Word_Merge.csv")
df_word = df_word.head(100000)
df_word

In [ ]:
set_word = set(df_word.iloc[:,0])

In [ ]:
set_spec_knee = set(df_spec_knee.iloc[:,0])

In [ ]:
df_word_spec = pd.DataFrame(set_word.intersection(set_spec_knee))
df_word_spec.rename(columns={0:"word"}, inplace=True)
df_word_spec

In [ ]:
df_word_spec.to_excel("Special_Name_In_Adjust_Word.xlsx", sheet_name="Spec_Name", index=False)